# Evaluate Data Quality

In this file we will try to find one data quality issue from the provided json files. The format will be in a series of quality related Q&A. Once we identify a problem, we will wrap up the exercise.

In [2]:
import pandas as pd
import os
import numpy as np

In [3]:
file = ['receipts', 'users', 'brands']

receipts = os.path.join('..', 'raw_data_files', file[0]+'.json')
users = os.path.join('..', 'raw_data_files', file[1]+'.json')
brands = os.path.join('..', 'raw_data_files', file[2]+'.json')

In [4]:
users

'../raw_data_files/users.json'

In [39]:
df_receipts = pd.read_json(receipts, orient='records', lines=True)
df_users = pd.read_json(users, orient='records', lines=True)
df_brands = pd.read_json(brands, orient='records', lines=True)

In [40]:
df_receipts.head(2)

,_id,bonusPointsEarned,bonusPointsEarnedReason,createDate,dateScanned,finishedDate,modifyDate,pointsAwardedDate,pointsEarned,purchaseDate,purchasedItemCount,rewardsReceiptItemList,rewardsReceiptStatus,totalSpent,userId
0,{'$oid': '5ff1e1eb0a720f0523000575'},500.0,"Receipt number 2 completed, bonus point schedu...",{'$date': 1609687531000},{'$date': 1609687531000},{'$date': 1609687531000},{'$date': 1609687536000},{'$date': 1609687531000},500.0,{'$date': 1609632000000},5.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,26.0,5ff1e1eacfcf6c399c274ae6
1,{'$oid': '5ff1e1bb0a720f052300056b'},150.0,"Receipt number 5 completed, bonus point schedu...",{'$date': 1609687483000},{'$date': 1609687483000},{'$date': 1609687483000},{'$date': 1609687488000},{'$date': 1609687483000},150.0,{'$date': 1609601083000},2.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,11.0,5ff1e194b6a9d73a3a9f1052


In [7]:
df_users.head(2)

,_id,active,createdDate,lastLogin,role,signUpSource,state
0,{'$oid': '5ff1e194b6a9d73a3a9f1052'},True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI
1,{'$oid': '5ff1e194b6a9d73a3a9f1052'},True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI


In [8]:
df_brands.head(2)

,_id,barcode,category,categoryCode,cpg,name,topBrand,brandCode
0,{'$oid': '601ac115be37ce2ead437551'},511111019862,Baking,BAKING,"{'$id': {'$oid': '601ac114be37ce2ead437550'}, ...",test brand @1612366101024,0.0,NaN
1,{'$oid': '601c5460be37ce2ead43755f'},511111519928,Beverages,BEVERAGES,"{'$id': {'$oid': '5332f5fbe4b03c9a25efd0ba'}, ...",Starbucks,0.0,STARBUCKS


## Question 1: Are the data types correct for each field?

In [9]:
df_receipts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1119 entries, 0 to 1118
Data columns (total 15 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   _id                      1119 non-null   object 
 1   bonusPointsEarned        544 non-null    float64
 2   bonusPointsEarnedReason  544 non-null    object 
 3   createDate               1119 non-null   object 
 4   dateScanned              1119 non-null   object 
 5   finishedDate             568 non-null    object 
 6   modifyDate               1119 non-null   object 
 7   pointsAwardedDate        537 non-null    object 
 8   pointsEarned             609 non-null    float64
 9   purchaseDate             671 non-null    object 
 10  purchasedItemCount       635 non-null    float64
 11  rewardsReceiptItemList   679 non-null    object 
 12  rewardsReceiptStatus     1119 non-null   object 
 13  totalSpent               684 non-null    float64
 14  userId                  

Nothing major worth to point out except the datatime objects. We will make sure when we analyze the data the datetime fields are not in a dictionary thus enabling us to be able to convert them.

In [12]:
df_users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 495 entries, 0 to 494
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   _id           495 non-null    object
 1   active        495 non-null    bool  
 2   createdDate   495 non-null    object
 3   lastLogin     433 non-null    object
 4   role          495 non-null    object
 5   signUpSource  447 non-null    object
 6   state         439 non-null    object
dtypes: bool(1), object(6)
memory usage: 23.8+ KB


Same thing here. Everything looks ok.

In [14]:
df_brands.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1167 entries, 0 to 1166
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   _id           1167 non-null   object 
 1   barcode       1167 non-null   int64  
 2   category      1012 non-null   object 
 3   categoryCode  517 non-null    object 
 4   cpg           1167 non-null   object 
 5   name          1167 non-null   object 
 6   topBrand      555 non-null    float64
 7   brandCode     933 non-null    object 
dtypes: float64(1), int64(1), object(6)
memory usage: 73.1+ KB


`topBrand` is supposed to be a Boolean field however after conversion from json to dataframes they are in the format of 0s and 1s. As long as there are no values other than 0 and 1 we are able to convert them into the correct format.

In [16]:
df_brands['topBrand'].value_counts()

0.0    524
1.0     31
Name: topBrand, dtype: int64

**Answer to Q1:**

Yes, the data types are correct for each field.
<hr>

## Question 2: Are the primary keys (supposedly) unique?

In [41]:
for index, row in df_receipts.iterrows():
    df_receipts.loc[index, '_id'] = df_receipts.loc[index, '_id']['$oid']

df_receipts.head()

,_id,bonusPointsEarned,bonusPointsEarnedReason,createDate,dateScanned,finishedDate,modifyDate,pointsAwardedDate,pointsEarned,purchaseDate,purchasedItemCount,rewardsReceiptItemList,rewardsReceiptStatus,totalSpent,userId
0,5ff1e1eb0a720f0523000575,500.0,"Receipt number 2 completed, bonus point schedu...",{'$date': 1609687531000},{'$date': 1609687531000},{'$date': 1609687531000},{'$date': 1609687536000},{'$date': 1609687531000},500.0,{'$date': 1609632000000},5.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,26.0,5ff1e1eacfcf6c399c274ae6
1,5ff1e1bb0a720f052300056b,150.0,"Receipt number 5 completed, bonus point schedu...",{'$date': 1609687483000},{'$date': 1609687483000},{'$date': 1609687483000},{'$date': 1609687488000},{'$date': 1609687483000},150.0,{'$date': 1609601083000},2.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,11.0,5ff1e194b6a9d73a3a9f1052
2,5ff1e1f10a720f052300057a,5.0,All-receipts receipt bonus,{'$date': 1609687537000},{'$date': 1609687537000},NaN,{'$date': 1609687542000},NaN,5.0,{'$date': 1609632000000},1.0,"[{'needsFetchReview': False, 'partnerItemId': ...",REJECTED,10.0,5ff1e1f1cfcf6c399c274b0b
3,5ff1e1ee0a7214ada100056f,5.0,All-receipts receipt bonus,{'$date': 1609687534000},{'$date': 1609687534000},{'$date': 1609687534000},{'$date': 1609687539000},{'$date': 1609687534000},5.0,{'$date': 1609632000000},4.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,28.0,5ff1e1eacfcf6c399c274ae6
4,5ff1e1d20a7214ada1000561,5.0,All-receipts receipt bonus,{'$date': 1609687506000},{'$date': 1609687506000},{'$date': 1609687511000},{'$date': 1609687511000},{'$date': 1609687506000},5.0,{'$date': 1609601106000},2.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,1.0,5ff1e194b6a9d73a3a9f1052


In [42]:
df_receipts['_id'].is_unique

True

Looking good for receipts...

In [43]:
# Let's look at users
for index, row in df_users.iterrows():
    df_users.loc[index, '_id'] = df_users.loc[index, '_id']['$oid']

df_users.head()

,_id,active,createdDate,lastLogin,role,signUpSource,state
0,5ff1e194b6a9d73a3a9f1052,True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI
1,5ff1e194b6a9d73a3a9f1052,True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI
2,5ff1e194b6a9d73a3a9f1052,True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI
3,5ff1e1eacfcf6c399c274ae6,True,{'$date': 1609687530554},{'$date': 1609687530597},consumer,Email,WI
4,5ff1e194b6a9d73a3a9f1052,True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI


In [44]:
df_users['_id'].is_unique

False

There are duplicated users in the `users` data. Let's find out who they are and how many.

In [48]:
print(f'There are {len(df_users)} users in total')

There are 495 users in total


In [50]:
dup_users = df_users.loc[df_users['_id'].duplicated()]
dup_users

,_id,active,createdDate,lastLogin,role,signUpSource,state
1,5ff1e194b6a9d73a3a9f1052,True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI
2,5ff1e194b6a9d73a3a9f1052,True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI
4,5ff1e194b6a9d73a3a9f1052,True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI
5,5ff1e194b6a9d73a3a9f1052,True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI
8,5ff1e194b6a9d73a3a9f1052,True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI
...,...,...,...,...,...,...,...
490,54943462e4b07e684157a532,True,{'$date': 1418998882381},{'$date': 1614963143204},fetch-staff,NaN,NaN
491,54943462e4b07e684157a532,True,{'$date': 1418998882381},{'$date': 1614963143204},fetch-staff,NaN,NaN
492,54943462e4b07e684157a532,True,{'$date': 1418998882381},{'$date': 1614963143204},fetch-staff,NaN,NaN
493,54943462e4b07e684157a532,True,{'$date': 1418998882381},{'$date': 1614963143204},fetch-staff,NaN,NaN


In [51]:
dup_users['role'].value_counts()

consumer       209
fetch-staff     74
Name: role, dtype: int64

In [100]:
# Filter out the fetch-staff
dup_users_consumer = dup_users.loc[dup_users['role'] != 'fetch-staff']
dup_users_consumer = dup_users_consumer.sort_values('_id')
dup_users_consumer.head(20)

,_id,active,createdDate,lastLogin,role,signUpSource,state
431,5a43c08fe4b014fd6b6a0612,True,{'$date': 1514389647059},{'$date': 1613146957155},consumer,NaN,NaN
428,5a43c08fe4b014fd6b6a0612,True,{'$date': 1514389647059},{'$date': 1613146957155},consumer,NaN,NaN
426,5a43c08fe4b014fd6b6a0612,True,{'$date': 1514389647059},{'$date': 1613146957155},consumer,NaN,NaN
425,5a43c08fe4b014fd6b6a0612,True,{'$date': 1514389647059},{'$date': 1613146957155},consumer,NaN,NaN
424,5a43c08fe4b014fd6b6a0612,True,{'$date': 1514389647059},{'$date': 1613146957155},consumer,NaN,NaN
423,5a43c08fe4b014fd6b6a0612,True,{'$date': 1514389647059},{'$date': 1613146957155},consumer,NaN,NaN
430,5a43c08fe4b014fd6b6a0612,True,{'$date': 1514389647059},{'$date': 1613146957155},consumer,NaN,NaN
429,5fb0a078be5fc9775c1f3945,True,{'$date': 1605410936818},NaN,consumer,Google,AL
19,5ff1e194b6a9d73a3a9f1052,True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI
15,5ff1e194b6a9d73a3a9f1052,True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI


Is there a good enough reason for there to be duplicated users / user IDs? One possible reason is that the `lastLogin` field documents a different login time for each user, however that almost sounds not feasible since it would make this "users" table extremely large and cumbersome, also it defeats the purpose if this is meant to be a customer database where we can access each unique account's basic customer information. More strongly, the definition for `lastLogin` provided in the assignment is: last time the user was recorded logging in to the app. As a result, it is likely that the same information is somehow recorded multiple times in the dataset.

We can see that the lastLogin field for the same customer contains the same datetime, but let's code it out to see the actual datetimes and compare.

In [102]:
for index, row in dup_users_consumer.loc[dup_users_consumer['lastLogin'].notna()].iterrows():
    dup_users_consumer.loc[index, 'lastLogin'] = dup_users_consumer.loc[index, 'lastLogin']['$date']

dup_users_consumer.head()

,_id,active,createdDate,lastLogin,role,signUpSource,state
431,5a43c08fe4b014fd6b6a0612,True,{'$date': 1514389647059},1613146957155,consumer,NaN,NaN
428,5a43c08fe4b014fd6b6a0612,True,{'$date': 1514389647059},1613146957155,consumer,NaN,NaN
426,5a43c08fe4b014fd6b6a0612,True,{'$date': 1514389647059},1613146957155,consumer,NaN,NaN
425,5a43c08fe4b014fd6b6a0612,True,{'$date': 1514389647059},1613146957155,consumer,NaN,NaN
424,5a43c08fe4b014fd6b6a0612,True,{'$date': 1514389647059},1613146957155,consumer,NaN,NaN


In [112]:
# Convert `lastLogin` to datetime
dup_users_consumer['lastLogin'] = pd.to_datetime(dup_users_consumer['lastLogin'], unit='ms')
dup_users_consumer.head()

,_id,active,createdDate,lastLogin,role,signUpSource,state
431,5a43c08fe4b014fd6b6a0612,True,{'$date': 1514389647059},2021-02-12 16:22:37.155,consumer,NaN,NaN
428,5a43c08fe4b014fd6b6a0612,True,{'$date': 1514389647059},2021-02-12 16:22:37.155,consumer,NaN,NaN
426,5a43c08fe4b014fd6b6a0612,True,{'$date': 1514389647059},2021-02-12 16:22:37.155,consumer,NaN,NaN
425,5a43c08fe4b014fd6b6a0612,True,{'$date': 1514389647059},2021-02-12 16:22:37.155,consumer,NaN,NaN
424,5a43c08fe4b014fd6b6a0612,True,{'$date': 1514389647059},2021-02-12 16:22:37.155,consumer,NaN,NaN


In [120]:
# This is an example of duplicated user having the exact same login timestamp (as well as other fields)
dup_users_consumer.loc[dup_users_consumer['_id'] == '5a43c08fe4b014fd6b6a0612']

,_id,active,createdDate,lastLogin,role,signUpSource,state
431,5a43c08fe4b014fd6b6a0612,True,{'$date': 1514389647059},2021-02-12 16:22:37.155,consumer,NaN,NaN
428,5a43c08fe4b014fd6b6a0612,True,{'$date': 1514389647059},2021-02-12 16:22:37.155,consumer,NaN,NaN
426,5a43c08fe4b014fd6b6a0612,True,{'$date': 1514389647059},2021-02-12 16:22:37.155,consumer,NaN,NaN
425,5a43c08fe4b014fd6b6a0612,True,{'$date': 1514389647059},2021-02-12 16:22:37.155,consumer,NaN,NaN
424,5a43c08fe4b014fd6b6a0612,True,{'$date': 1514389647059},2021-02-12 16:22:37.155,consumer,NaN,NaN
423,5a43c08fe4b014fd6b6a0612,True,{'$date': 1514389647059},2021-02-12 16:22:37.155,consumer,NaN,NaN
430,5a43c08fe4b014fd6b6a0612,True,{'$date': 1514389647059},2021-02-12 16:22:37.155,consumer,NaN,NaN


Result: **The primary key for the "users" dataset is not unique.**

In [121]:
# How about the brands id?
for index, row in df_brands.iterrows():
    df_brands.loc[index, '_id'] = df_brands.loc[index, '_id']['$oid']

df_brands.head()

,_id,barcode,category,categoryCode,cpg,name,topBrand,brandCode
0,601ac115be37ce2ead437551,511111019862,Baking,BAKING,"{'$id': {'$oid': '601ac114be37ce2ead437550'}, ...",test brand @1612366101024,0.0,NaN
1,601c5460be37ce2ead43755f,511111519928,Beverages,BEVERAGES,"{'$id': {'$oid': '5332f5fbe4b03c9a25efd0ba'}, ...",Starbucks,0.0,STARBUCKS
2,601ac142be37ce2ead43755d,511111819905,Baking,BAKING,"{'$id': {'$oid': '601ac142be37ce2ead437559'}, ...",test brand @1612366146176,0.0,TEST BRANDCODE @1612366146176
3,601ac142be37ce2ead43755a,511111519874,Baking,BAKING,"{'$id': {'$oid': '601ac142be37ce2ead437559'}, ...",test brand @1612366146051,0.0,TEST BRANDCODE @1612366146051
4,601ac142be37ce2ead43755e,511111319917,Candy & Sweets,CANDY_AND_SWEETS,"{'$id': {'$oid': '5332fa12e4b03c9a25efd1e7'}, ...",test brand @1612366146827,0.0,TEST BRANDCODE @1612366146827


In [123]:
df_brands['_id'].is_unique

True

**Answer to Q2:**

As there are no obvious scenarios for dual or more primary keys in the users dataset, the fact that there are duplicated IDs in the dataset indicates a potential data integrity issue.

<hr>

## End